In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
img = plt.imread('MazeS_10x10.png')
maze_size = 10
maze_px = 1
start = (0,0)
end = (4,4)

maze_viz = img.copy()
for y in xrange(start[0]*maze_px,(start[0]+1)*(maze_px)):
    for x in xrange(start[1]*maze_px,(start[1]+1)*(maze_px)):
        maze_viz[y,x,:] = 0.4
for y in xrange(end[0]*maze_px,(end[0]+1)*(maze_px)):
    for x in xrange(end[1]*maze_px,(end[1]+1)*(maze_px)):
        maze_viz[y,x,:] = 0.0
plt.imshow(maze_viz,interpolation='nearest')

In [ ]:
up = (0.75,0,0)
down = (0.25,0,0.0)
left = (0.0,0.75,0)
right = (0.0,0.25,0)

sol = np.zeros_like(img)
dist = np.zeros_like(img)

visited = np.zeros_like(img)
visited[maze_viz[:,:,0] > 0.5] = 1

queue = []
for y in xrange(img.shape[0]):
    for x in xrange(img.shape[1]):
        if maze_viz[y,x,:].sum() == 0:
            queue.append([y,x,0,0])
            visited[y,x,:] = 1
steps = 0
while len(queue) > 0:
    y,x,c,s = queue[0]
    queue = queue[1:]
    if y+1 < img.shape[0] and visited[y+1,x,0] < 1:
        queue.append([y+1,x,down,s+1])
    if x+1 < img.shape[1] and visited[y,x+1,0] < 1:
        queue.append([y,x+1,right,s+1])
    if y-1 >= 0 and visited[y-1,x,0] < 1:
        queue.append([y-1,x,up,s+1])
    if x-1 >= 0 and visited[y,x-1,0] < 1:
        queue.append([y,x-1,left,s+1])
    sol[y,x,:] = c
    dist[y,x,:] = s
    visited[y,x,:] = 1
plt.imshow(sol,interpolation='nearest')

In [ ]:
dist /= dist.max()
plt.subplot(1,3,1)
plt.imshow(maze_viz,interpolation='nearest')
plt.xticks([])
plt.title('maze')
plt.yticks([])
plt.subplot(1,3,2)
plt.imshow(dist,interpolation='nearest')
plt.xticks([])
plt.title('distance to goal')
plt.yticks([])
plt.subplot(1,3,3)
plt.imshow(sol,interpolation='nearest')
plt.title('solution (g: l/r, r:u/d)')
plt.xticks([])
plt.yticks([])
plt.tight_layout()